In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import networkx as nx
from karateclub import Graph2Vec
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import torch
from torch_geometric.utils import to_networkx, from_networkx
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import umap


In [ ]:
cases_graphs = []
# Parse the generated cases network
for i in range(50):
    graph_data_obj_seed150_path = "/grehawi/splice-reg-prj/new-data/Graph-validation-ARROW-Diff/ARROW-Diff-output/cases-net/graphs_seed150/"
    graph_data_obj_seed150_path = graph_data_obj_seed150_path + "graph_" + str(i+1) + ".pt"
    graph_data_obj_seed150 = torch.load(graph_data_obj_seed150_path)
    graph = to_networkx(graph_data_obj_seed150, to_undirected=True)
    cases_graphs.append(graph)
    
for i in range(50):
    graph_data_obj_seed149_path = "/grehawi/splice-reg-prj/new-data/Graph-validation-ARROW-Diff/ARROW-Diff-output/cases-net/graphs_seed149/"
    graph_data_obj_seed149_path = graph_data_obj_seed149_path + "graph_" + str(i+1) + ".pt"
    graph_data_obj_seed149 = torch.load(graph_data_obj_seed149_path)
    graph = to_networkx(graph_data_obj_seed149, to_undirected=True)
    cases_graphs.append(graph)


In [ ]:
controls_graphs = []
# Parse the generated controls network
for i in range(50):
    graph_data_obj_seed150_path = "/grehawi/splice-reg-prj/new-data/Graph-validation-ARROW-Diff/ARROW-Diff-output/controls-net/graphs_seed150/"
    graph_data_obj_seed150_path = graph_data_obj_seed150_path + "graph_" + str(i+1) + ".pt"
    graph_data_obj_seed150 = torch.load(graph_data_obj_seed150_path)
    graph = to_networkx(graph_data_obj_seed150, to_undirected=True)
    controls_graphs.append(graph)
    
for i in range(50):
    graph_data_obj_seed149_path = "/grehawi/splice-reg-prj/new-data/Graph-validation-ARROW-Diff/ARROW-Diff-output/controls-net/graphs_seed149/"
    graph_data_obj_seed149_path = graph_data_obj_seed149_path + "graph_" + str(i+1) + ".pt"
    graph_data_obj_seed149 = torch.load(graph_data_obj_seed149_path)
    graph = to_networkx(graph_data_obj_seed149, to_undirected=True)
    controls_graphs.append(graph)


In [ ]:
cases_graphs.extend(controls_graphs)
model = Graph2Vec(dimensions = 128)

model.fit(cases_graphs)
graphs_embed = model.get_embedding()

In [ ]:
graphs_embed.shape

## PCA on the graphs embeddings

In [ ]:
pca = PCA(n_components=5)
X_r = pca.fit(graphs_embed).transform(graphs_embed)

In [ ]:
print(pca.explained_variance_ratio_)

In [ ]:
PC1 = X_r[:,0]
PC2 = X_r[:,1]
labels = ["Affected Individuals Networks" for i in range(100)]
labels.extend(["Unaffected Individuals Networks" for i in range(100)])
zipped = list(zip(PC1, 
                  PC2, 
                  labels))

pc_df = pd.DataFrame(zipped, 
                     columns=['PC1', 
                              'PC2',
                              'labels'])
plt.figure(figsize=(10, 7))
 
sns_plot = sns.scatterplot(data=pc_df, 
                x="PC1", 
                y="PC2",
                hue = labels,
                palette=['#F0CF7F', '#5DB9B5'])
 
plt.legend(loc='upper left', bbox_to_anchor=(1, 0.5), ncol=1, shadow=False)
plt.legend(fontsize=16)
plt.xlabel('PC1 (4%)',
             fontsize=16)
plt.ylabel('PC2 (3%)',
             fontsize=16)
plt.show()
#sns_plot.figure.savefig("/grehawi/splice-reg-prj/Figures/pca-arrowDiff-graphs-embedding-graph2vec-128dim.png", dpi=300)

## Classifying the embeddings

In [ ]:
from sklearn.linear_model import LinearRegression
X = graphs_embed
# y = 100x1 (case class)+ 100x0 (controls class) 
ones = np.ones(100)
zeros = np.zeros(100)
y = np.concatenate((ones, zeros))
reg = LinearRegression().fit(X, y)
reg.score(X, y)

In [ ]:
reg.intercept_